# OuterSpace

In [1]:
# HiFiber boilerplate

from fibertree_bootstrap import *

fibertree_bootstrap(style="tree", animation='movie')

# Compilation boilerplate

import os
import sys
sys.path.insert(0, "..")

from src.utils import *

Running bootstrap
The fibertree module is already installed and available to import


interactive(children=(Dropdown(description='style', options=('tree', 'uncompressed', 'tree+uncompressed'), val…

Button(description='Run all cells below', style=ButtonStyle())

In [2]:
K = 5
M = 6
N = 4

seed = 0

A_KM = Tensor.fromRandom(rank_ids=["K", "M"], shape=[K, M], seed=seed, density=[0.75, 0.5], name="A")
B_KN = Tensor.fromRandom(rank_ids=["K", "N"], shape=[K, N], seed=seed + 1, density=[0.75, 0.5], name="B")

In [3]:
# Autogenerated HiFiber

T_KM2KM1KM0N = Tensor(rank_ids=["KM2", "KM1", "KM0", "N"], name="T")
t_km2 = T_KM2KM1KM0N.getRoot()
tmp0 = A_KM
tmp1 = tmp0.flattenRanks(depth=0, levels=1, coord_style="tuple")
A_KM_flat = tmp1
A_KM_flat.setRankIds(rank_ids=["KM"])
b_k = B_KN.getRoot()
a_km = A_KM_flat.getRoot()
canvas = createCanvas(A_KM_flat, B_KN, T_KM2KM1KM0N)
A_KM_flat = Tensor.fromFiber(rank_ids=["KM"], fiber=a_km)
tmp2 = A_KM_flat
tmp3 = tmp2.splitEqual(256)
A_KM2KM1I = tmp3
A_KM2KM1I.setRankIds(rank_ids=["KM2", "KM1I"])
a_km2 = A_KM2KM1I.getRoot()
for km2_pos, (km2, (t_km1, a_km1i)) in enumerate(t_km2 << a_km2):
    A_KM1I = Tensor.fromFiber(rank_ids=["KM1I"], fiber=a_km1i)
    tmp4 = A_KM1I
    tmp5 = tmp4.splitEqual(16)
    A_KM1KM0 = tmp5
    A_KM1KM0.setRankIds(rank_ids=["KM1", "KM0"])
    a_km1 = A_KM1KM0.getRoot()
    for km1_pos, (km1, (t_km0, a_km0)) in enumerate(t_km1 << a_km1):
        for km0_pos, ((k, m), (t_n, a_val)) in enumerate(t_km0 << a_km0):
            b_n = b_k.getPayload(k)
            for n_pos, (n, (t_ref, b_val)) in enumerate(t_n << b_n):
                t_ref += a_val * b_val
                canvas.addActivity(((k, m),), (k, n), (km2, km1, (k, m), n), spacetime=((km1_pos, km0_pos), (km2_pos, n_pos)))
tmp6 = T_KM2KM1KM0N
tmp7 = tmp6.swizzleRanks(rank_ids=["N", "KM2", "KM1", "KM0"])
tmp8 = tmp7.mergeRanks(depth=1, levels=2, coord_style="absolute")
tmp9 = tmp8.unflattenRanks(depth=1, levels=1)
tmp9.setRankIds(rank_ids=["N", "K", "M"])
tmp10 = tmp9.swizzleRanks(rank_ids=["K", "M", "N"])
T_KMN = tmp10
displayCanvas(canvas)

{(0, 4): [(3, 0)]}
{(0, 4): [(3, 0)]}
{(0, 4): [(0, 0), (0, 0), (3, 0), 0]}


TypeError: '<' not supported between instances of 'tuple' and 'int'

In [ ]:
yaml = """
einsum:
    declaration:
        A: [K, M]
        B: [K, N]
        T: [K, M, N]
        Z: [M, N]
    expressions:
        - T[k, m, n] = A[k, m] * B[k, n]
        - Z[m, n] = T[k, m, n]
mapping:
    rank−order:
        A: [K, M]
        B: [K, N]
        T: [M, K, N]
        Z: [M, N]
    partitioning:
        T:
            (K, M): [flatten()]
            KM: [uniform_occupancy(A.256), uniform_occupancy(A.16)]
        Z:
            M: [uniform_occupancy(T.128), uniform_occupancy(T.8)]
    loop−order:
        T: [KM2, KM1, KM0, N]
        Z: [M2, M1, M0, N, K]
    spacetime:
        T:
            space: [KM1, KM0]
            time: [KM2, N]
        Z:
            space: [M1, M0]
            time: [M2, N, K]
"""

compile(yaml)